### Preprocess Data

In [1]:
import pandas as pd
data = pd.read_csv("19hz_with_embeddings.csv",index_col=0)
data2 = pd.read_csv("funcheap_with_embeddings.csv",index_col=0)
data3 = pd.read_csv("eventbrite_with_embeddings.csv",index_col=0)

In [2]:
data3['price'] = 'Free'
data3.head(1)

,venue,event,date,age,price,location,event_description,geolocation,venue_description,neighborhood,combined,n_tokens,embedding
0,"Celo • San Francisco, CA",DwellFi PropTech and Web3 Event,"Thu, February 23, 2023, 5:30 PM – 8:30 PM PST",NaN,Free,"Celo 500 Treat Avenue San Francisco, CA 94110",T﻿his event is focused on PropTech and Web3.Pa...,POINT (37.735286 -122.408126),Celo is a contemporary art gallery located in ...,The venue is located in the Lower Haight distr...,"Venue: Celo • San Francisco, CA; Venue Descrip...",809,"[0.015124982222914696, -0.024480924010276794, ..."


In [5]:
data.head(2)

,venue,event,date,age,price,location,geolocation,venue_description,neighborhood,combined,n_tokens,embedding,event_description
0,1015 Folsom (San Francisco),Good Fun Luv n More,Wed: Feb 22 (6pm-10pm),21+,$10-15,"1015 Folsom Street\nSan Francisco, CA 94103",POINT (37.7780411 -122.405683),The 1015 Folsom is a nightclub located in San ...,The venue is located in the SoMa district of S...,Venue: 1015 Folsom (San Francisco); Venue Desc...,88,"[-0.001874412759207189, -0.0360201820731163, -...",['house']['The Good Fun Event Link']
1,Mothership Bar,Local / Global,Wed: Feb 22 (8pm-12am),21+,free,The Mothership Bar is located at 333 Bush Stre...,POINT (32.38141 76.51194),Mothership Bar is a speakeasy-style bar locate...,The Mothership Bar is located in the Haight-As...,Venue: Mothership Bar; Venue Description: Moth...,99,"[0.0026790155097842216, -0.03984867408871651, ...","['house', 'techno', 'global beats']['DJmaDRE']"


In [4]:
data['event_description'] = data['music'] + data['performers']
data = data.drop(['music','performers'],axis=1)

In [6]:
new_order = ['venue', 'event', 'date', 'age', 'price','location','geolocation','venue_description', 'event_description', 'neighborhood','combined','n_tokens','embedding']
# Reorder the columns using the reindex() method
data2 = data2.rename(columns={'event_name':'event'})
data2 = data2.reindex(columns=new_order)
data3 = data3.reindex(columns=new_order)
data = data.reindex(columns=new_order)
data.tail(1)


,venue,event,date,age,price,location,geolocation,venue_description,event_description,neighborhood,combined,n_tokens,embedding
188,Audio (San Francisco),"Ewan McVicar, Maj. Blk&Wht",Fri: Mar 24 (9:30pm-2:30am),21+,$11 pre,Audio (San Francisco)\n\n415 Geary Street\n\nS...,POINT (37.779278 -122.416582),Audio is a music venue located in the heart of...,"['techno', 'house']['All Day All Night Events']",The Audio in San Francisco is located in the M...,Venue: Audio (San Francisco); Venue Descriptio...,123,"[0.00950961746275425, -0.03634786978363991, -0..."


In [7]:
data2.head(1)

,venue,event,date,age,price,location,geolocation,venue_description,event_description,neighborhood,combined,n_tokens,embedding
0,Cobb's Comedy Club,Secret Guest List: Cobb’s Free Comedy Night (F...,"Wednesday, February 22, 2023 - 8:00 pm to 10:0...",18 & over with valid photo ID,Free with RSVP,"915 Columbus Ave, San Francisco, CA 94133North...",POINT (37.77493 -122.41942),Cobb's Comedy Club is a live comedy club in Sa...,Funcheap and HellaFunny are taking over Cobb's...,The venue is located in the Financial District...,Venue: Cobb's Comedy Club; Venue Description: ...,240,"[0.006269877776503563, -0.03384096920490265, 0..."


In [8]:
data[['venue', 'event', 'date', 'age', 'price','location','geolocation','venue_description', 'event_description', 'neighborhood','combined']] = data[['venue', 'event', 'date', 'age', 'price','location','geolocation','venue_description', 'event_description', 'neighborhood','combined']].astype(str)
data2[['venue', 'event', 'date', 'age', 'price','location','geolocation','venue_description', 'event_description', 'neighborhood','combined']] = data2[['venue', 'event', 'date', 'age', 'price','location','geolocation','venue_description', 'event_description', 'neighborhood','combined']].astype(str)
data3[['venue', 'event', 'date', 'age', 'price','location','geolocation','venue_description', 'event_description', 'neighborhood','combined']] = data3[['venue', 'event', 'date', 'age', 'price','location','geolocation','venue_description', 'event_description', 'neighborhood','combined']].astype(str)

In [9]:
concatenated_data = pd.concat([data,data,data3])
df = concatenated_data.loc[concatenated_data['geolocation'] != 'nan']

In [10]:
from math import radians, sin, cos, sqrt, atan2
from shapely.geometry import Point
from shapely.wkt import loads


my_loc = Point(37.7652844,-122.4329059)


def distance_in_miles(loc1, loc2):

    loc2 = loads(loc2)
    """Calculate the distance in miles between two geolocation points using the Haversine formula."""
    # Convert coordinates to radians
    lon1, lat1 = radians(loc1.x), radians(loc1.y)
    lon2, lat2 = radians(loc2.x), radians(loc2.y)


    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance_miles = 3963.0 * c # Radius of Earth in miles

    return float(round(distance_miles,3))

df['distance_miles'] = df.apply(lambda row: distance_in_miles(my_loc, row['geolocation']), axis=1)



/var/folders/z0/hz8lx2755md0tlm5d20wdrlw0000gn/T/ipykernel_31988/1416835756.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['distance_miles'] = df.apply(lambda row: distance_in_miles(my_loc, row['geolocation']), axis=1)


In [11]:
df = df.drop('geolocation', axis=1)

In [12]:
df.to_csv('preprocessed_data.csv')